### Boot Finder 

In [ ]:
# If this is your first time in the JupyterLab workspace - install external dependencies
from utilities.dependencies import install
install(timeout=500)

# No need to do this in the future for this notebook

In [ ]:
# Import necessary libraries
import os
import urllib

import coremltools 
from skafossdk import *
import turicreate as tc
import pandas as pd
import numpy as np
from s3fs.core import S3FileSystem

In [ ]:
# Initialize Skafos
ska = Skafos()

In [ ]:
#Pull in boot images from S3 
_local_dir = 'BootImages'

if not os.path.exists(_local_dir):
    os.makedirs(_local_dir)

# Connect to S3
s3 = S3FileSystem(anon=True)

# Bring in boot data
boots = s3.ls("s3://mf.testing.bucket/Boots/Images/")
        
# Loop over the image paths, adding them to the SFrames
for b in boots: 
    _local_file = "/".join(b.split("/")[-1:])
    _local_path = _local_dir + "/" + _local_file
    s3.get("s3://" + b, _local_path) # download the image

In [ ]:
# Pull in boot metadata from S3
csv = "s3://mf.testing.bucket/Boots/BootMetadata_20190302.csv"
print("/".join(csv.split("/")[-1:]))
s3.get(csv, "/".join(csv.split("/")[-1:]))

In [ ]:
# Pull in F21 json boot data and metadata
json_path = "s3://mf.testing.bucket/Boots/F21_shoes.json"
df = pd.read_json(json_path)
components = pd.DataFrame.from_dict(df.iloc[8].body)
F21_meta = pd.DataFrame.from_records(components.docs)
F21_meta['path'] = F21_meta['thumb_image'].apply(lambda x: 'BootImages/' + x.split("/")[-2:][1])

for index, row in F21_meta.iterrows():
    urllib.request.urlretrieve(row['thumb_image'], row['path'])
    
F21_meta = F21_meta.rename(index=str, columns={'title': 'Description'})
F21_meta['Image Name'] = F21_meta['path'].apply(lambda x: x.split("/")[-2:][1])

In [ ]:
# Pull in ebay boot data and metadata
json_path = "s3://mf.testing.bucket/Boots/ebay_boots.json"
df = pd.read_json(json_path)
components_df = pd.DataFrame.from_records(df.itemSummaries)

ebay_metadata = pd.DataFrame()
ebay_metadata['thumb_image'] = pd.DataFrame.from_records(components_df.image)['imageUrl']
ebay_metadata['price'] = pd.DataFrame.from_records(components_df.price)['value'].astype(float)
ebay_metadata['url'] = components_df.itemWebUrl
ebay_metadata['Description'] = components_df.title
ebay_metadata['path'] = ebay_metadata['thumb_image'].apply(lambda x: 'BootImages/' + x.split("/")[-2] + x.split("/")[-1])
ebay_metadata['Image Name'] = ebay_metadata['thumb_image'].apply(lambda x: x.split("/")[-2] + x.split("/")[-1])

for index, row in ebay_metadata.iterrows():
    urllib.request.urlretrieve(row['thumb_image'], row['path'])

In [ ]:
# Combine F21 and ebay metadata
meta_combined = pd.merge(F21_meta, ebay_metadata, 
                         on=['Image Name','path', 'Description', 'url', 'price', 'thumb_image'], how='outer')

In [ ]:
# Combine metadata
meta_combined = meta_combined.astype(str).replace(np.nan, 'None')
meta_combined_sframe = tc.SFrame(meta_combined)
metadata = tc.SFrame.read_csv("BootMetadata_20190302.csv")
metadata['path'] = metadata['Image Name'].apply(lambda x: 'BootImages/' + x)
metadata = metadata.join(meta_combined_sframe, on=["Image Name", "Description", "path"], how="outer")

In [ ]:
# Create boot SFrame
boot_data  = tc.image_analysis.load_images('BootImages')

In [ ]:
metadata.export_csv("AllMetadata.csv")

In [ ]:
#Join create image_name to merge SFrames
boot_data['Image Name'] = boot_data['path'].apply(lambda x: x.split("/")[-2:][1])
boot_data = boot_data.join(metadata, on=["Image Name", "path"], how="left")
boot_data = boot_data.add_row_number()

In [ ]:
boot_data.shape

In [ ]:
model = tc.image_similarity.create(boot_data, label="Description")

In [ ]:
img = tc.Image("IMG_3436.JPG")
model.query(img)

In [ ]:
# Specify the CoreML model name
model_name = 'ImageSimilarity'
coreml_model_name = model_name + '.mlmodel'

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name) 

# Use coremltools to convert model weights to half-precision.
# This may be necessary if you have memory concerns within your app
#model_spec = coremltools.utils.load_spec(coreml_model_name)
#model_fp16_spec = coremltools.utils.convert_neural_network_spec_weights_to_fp16(model_spec)
#coremltools.utils.save_spec(model_fp16_spec, coreml_model_name)

# Save model asset to Skafos
ska.asset_manager.save(
    name=model_name,              # Name used to load or deliver asset, also used within the Swift SDK.
    files=coreml_model_name,      # File or list of files to bundle together as a versioned asset.
    tags=['latest'],              # User-defined tags to help distinguish your asset.
    access='public'               # Asset access- public/private.
)

In [ ]:
# Deliver asset to devices (push)
ska.asset_manager.deliver(
  name=model_name,                # Name used to load or deliver asset, also used within the Swift SDK.
  tag='latest',                   # User-defined tags to help distinguish your asset.
  dev=True                        # Push asset through Apple's APNS dev or prod server
)